# FreeGSNKE Static Exmaples
## Inverse Solve and Forward Solve for Grad-Shafranov Equilibria

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import sys
import matplotlib.pyplot as plt
from copy import deepcopy
from IPython.display import display, clear_output
import time
import os

In [ ]:
import freegs
from freegs.plotting import plotConstraints
from freegs.critical import find_critical
from freegsnke import machine_config
from freegsnke import build_machine
from freegsnke import faster_shape
from freegsnke.jtor_update import ConstrainPaxisIp
from freegsnke import GSstaticsolver

%load_ext autoreload
%autoreload 2

## Create the machine.

First, identify files containing the machine description. FreeGSNKE requires the user to define the following environment variables:

ACTIVE_COILS_PATH , PASSIVE_COILS_PATH , WALL_PATH , LIMITER_PATH . These define the machine geometry and properties. 

Right now, they are saved as pickle dictionaries, mostly for convenience because the active-coil dictionary contains a lot of info about the coil windings.

In [ ]:
# Create the machine.

# First, identify files containing the machine description.
# freeGSNKE requires the user to define the following environment variables 
# ACTIVE_COILS_PATH
# PASSIVE_COILS_PATH
# WALL_PATH
# LIMITER_PATH
# These define the machine geometry and properties.
import os
os.environ["ACTIVE_COILS_PATH"] = "../machine_configs/MAST-U/active_coils.pickle"
os.environ["PASSIVE_COILS_PATH"] = "../machine_configs/MAST-U/passive_coils.pickle"
os.environ["WALL_PATH"] = "../machine_configs/MAST-U/wall.pickle"
os.environ["LIMITER_PATH"] = "../machine_configs/MAST-U/limiter.pickle"

# Now the machine can actually be built:
from freegsnke import build_machine
tokamak = build_machine.tokamak()



In [ ]:
# Create the equilibrium and profile objects

# This creates the equilibrium object. 
# The equilibrium object indirectly defines the rectangular domain of the solver,
# as well as the grid resolution. 
eq = freegs.Equilibrium(tokamak=tokamak, #this assigns the machine just created above to the equilibrium
                        # the following define the rectangular domain
                        Rmin=0.1, Rmax=2.0,    # Radial range
                        Zmin=-2.2, Zmax=2.2,   # Vertical range
                        #grid resolution 
                        nx=65, # Number of grid points in the radial direction
                        ny=129, # Number of grid points in the vertical direction
                        # If available, an initial plasma flux function "plasma_psi" can be provided:
                        # psi=initial guess plasma_psi
                        )  


# Creates the profile object. 
# The profile object sets the parametrization and properties of the profile function. 
# freeGSNKE accepts the same profile classes available in freeGS: ConstrainPaxisIp or ConstrainBetapIp.
# The profile profile object also sets the value of the total plasma current.
# Example use of a ConstrainPaxisIp profile object:
from freegsnke.jtor_update import ConstrainPaxisIp
profiles = ConstrainPaxisIp(# Equilibrium and limiter objects are passed to the profile to inform calculations relating to the limiter
                            eq, tokamak.limiter,
                            # The following set the desired values for the profile properties
                            8.1e3, # Plasma pressure on axis [Pascals]
                            6.2e5, # Total plasma current [Amps]
                            0.5, # vacuum f = R*Bt
                            # The following are the coefficients that define the peakedness of the plasma current density distribution
                            # see e.g. arXiv:1503.03135
                            alpha_m = 1.8,
                            alpha_n = 1.2)
# For an example use of a ConstrainBetapIp profile object, please un-comment the following:
# from freegsnke.jtor_update import ConstrainBetapIp
# profiles = ConstrainBetapIp(
#                               # Equilibrium and limiter objects are passed to the profile to inform calculations relating to the limiter:
#                               eq, tokamak.limiter,
#                               # The following set the desired values for the profile properties
#                               0.2, # value of beta_poloidal
#                               6.2e5, # Plasma current [Amps]
#                               0.5, # vacuum f = R*Bt
#                               # The following are the coefficients that define the peakedness of the plasma current density distribution
#                               # see e.g. arXiv:1503.03135
#                               alpha_m = 1.8,
#                               alpha_n = 1.2)

## Inverse solve and forward solve for a diverted equilibrium

We start with an equilibrium where the plasma is entirely contained by its last-closed-flux-surface (LCFS), which goes through at least one of the X-points (saddle-points in $\psi$) and does not intersect the tokamak's limiter. We start with an _inverse_ solve, where we specify a few points that must lie on the same LCFS, and the coil currents are then determined together with $\psi(R,Z)$. We then show how to do a _forward_ solve, for which we need to specify the currents instead, and the profile $\psi(R,Z)$ is to be determined. The invere solve alternates a Picard iteration ($\psi^{(n+1)}=(\Delta^*)^{-1}\psi^{(n)}$) with one adjustment in the coil currents such that the resulting LCFS is as close as possible to the prescribed constraints, iterating until convergence. The forward solver uses the Newton-Krylov implementation described by Amorisco et al (2024) in _Physics of Plasmas_.

In [ ]:
# Inverse-solve of an equilibrium 


In [ ]:

# Creates equilibrium object and initializes it with 
# a "good" solution
# plasma_psi = np.loadtxt('plasma_psi_example.txt')
eq = freegs.Equilibrium(tokamak=tokamak,
                        #domains can be changed 
                        Rmin=0.1, Rmax=2.0,    # Radial domain
                        Zmin=-2.2, Zmax=2.2,   # Height range
                        #grid resolution can be changed
                        nx=65, ny=129, # Number of grid points
                        # psi=plasma_psi[::2,:])   
                        )  



# Sets desired plasma properties for the 'starting equilibrium'
# Use one between ConstrainPaxisIp or ConstrainBetapIp
# values can be changed
from freegsnke.jtor_update import ConstrainPaxisIp
profiles = ConstrainPaxisIp(eq, tokamak.limiter,
                            8.1e3, # Plasma pressure on axis [Pascals]
                            6.2e5, # Plasma current [Amps]
                            0.5, # vacuum f = R*Bt
                            alpha_m = 1.8,
                            alpha_n = 1.2)
# from freegsnke.jtor_update import ConstrainBetapIp
# profiles = ConstrainBetapIp(eq, tokamak.limiter,
#                             0.2, # Plasma pressure on axis [Pascals]
#                             6.2e5, # Plasma current [Amps]
#                             0.5, # vacuum f = R*Bt
#                             alpha_m = 1.8,
#                             alpha_n = 1.2)


# Sets some shape constraints (here very close to those used for initialization)
Rx = 0.6
Zx = 1.1

Rmid = 1.41   # Outboard midplane
Rin = 0.36  # Inboard midplane

xpoints = [(Rx, -Zx-.01),   # (R,Z) locations of X-points
           (Rx,  Zx)]
isoflux = [
           (Rx,Zx, Rx,-Zx),
           (Rmid, 0, Rin, 0.0),
           (Rmid,0, Rx,Zx),
           (Rmid,0, 1.2,.7),
           (Rmid,0, 1.2,-.7),
    
           # Link inner and outer midplane locations
           (Rx, Zx, .85, 1.7),
           (Rx, Zx, .75, 1.6),
           (Rx, Zx, Rin, 0.2),
           (Rx, Zx, Rin, 0.1),
           (Rx,-Zx, Rin, -0.1),
           (Rx,-Zx, Rin, -0.2),
           (Rx,-Zx, .85, -1.7),
           (Rx,-Zx, .75, -1.6),

           (Rx,-Zx, 0.45, -1.8),
           (Rx, Zx, 0.45,  1.8),
           ]

eq.tokamak['P6'].current = 0
eq.tokamak['P6'].control = False
eq.tokamak['Solenoid'].control = False

constrain = freegs.control.constrain(xpoints=xpoints, 
                                     gamma=5e-6, 
                                     isoflux=isoflux
                                    )
constrain(eq)
                                    
from freegsnke import GSstaticsolver
NK = GSstaticsolver.NKGSsolver(eq)         

In [ ]:
eq.tokamak['P6'].current = 0
eq.tokamak['P6'].control = False
eq.tokamak['Solenoid'].control = False
eq.tokamak['Solenoid'].current = 15000
# Nonlinear solve
freegs.solve(eq,          # The equilibrium to adjust
             profiles,    # The plasma profiles
             constrain,   # Plasma control constraints
             show=False,
             rtol=3e-3)               
eq.tokamak['Solenoid'].current = 40000
freegs.solve(eq,          # The equilibrium to adjust
             profiles,    # The plasma profiles
             constrain,   # Plasma control constraints
             show=False,
             rtol=3e-3)  
NK.solve(eq, profiles, target_relative_tolerance=1e-8)
fig = plt.figure(figsize=(5, 10), dpi=80);
ax = fig.add_subplot(111);
ax.grid(True,which='both');
eq.plot(axis=ax,show=False);
eq.tokamak.plot(axis=ax,show=False);
constrain.plot(axis=ax,show=False)

qprof = eq.q()
plt.figure()
plt.plot(qprof[0], qprof[1], '+')
plt.title('q profile')

In [ ]:
# Forward-solve of an equilibrium

# Assign current values to the active poloidal field coils 
# corresponding to a diverted plasma. 
# This assigns the passive structure currents too, in this specific case these are all set to zero.
# Set initial equilibrium

# Assign current values to the active poloidal field coils 
# corresponding to a diverted plasma. 
# This assigns the passive structure currents too, in this specific case these are all set to zero.
import pickle
with open('simple_diverted_currents.pk', 'rb') as f:
    current_values = pickle.load(f)
for key in current_values.keys():
    eq.tokamak[key].current = current_values[key]

# Instantiate freeGSNKE static GS solver
from freegsnke import GSstaticsolver
# This requires use of the equilibrium itself to inform the solver of the domain and grid properties:
NK = GSstaticsolver.NKGSsolver(eq)
# Solve the forward GS problem corresponding to the tokamak metal currents set above and the requested profile properties
NK.solve(# The Equilibrium object sets the currents 
         eq, 
         # The profile sets the desired plasma properties
         profiles,
         # The relative tolerance set for convergence
         target_relative_tolerance=1e-8,
         verbose=1)

# Plot the equilibrium
# As in freeGS, the full black line is the tokamak.wall.
# In addition, the thin black dotted line shows the tokamak.limiter,
# i.e. the border of the region where the plasma is allowed to be.
# The limiter is passed to the equilibrium object by the profile object during the NK.solve call. 
fig = plt.figure(figsize=(5, 10), dpi=80)
ax = fig.add_subplot(111)
ax.grid(True,which='both')
eq.plot(axis=ax,show=False)
eq.tokamak.plot(axis=ax,show=False)



In [ ]:
### Then, we instantiate two objects: the "equilibrium" and its internal p' and f'f "profiles"

In [ ]:
# Create the equilibrium and profile objects

# This creates the equilibrium object. 
# The equilibrium object indirectly defines the rectangular domain of the solver,
# as well as the grid resolution. 
eq = freegs.Equilibrium(tokamak=tokamak, #this assigns the machine just created above to the equilibrium
                        # the following define the rectangular domain
                        Rmin=0.1, Rmax=2.0,    # Radial range
                        Zmin=-2.2, Zmax=2.2,   # Vertical range
                        #grid resolution 
                        nx=65, # Number of grid points in the radial direction
                        ny=129, # Number of grid points in the vertical direction
                        # If available, an initial plasma flux function "plasma_psi" can be provided:
                        # psi=initial guess plasma_psi
                        )  


# Creates the profile object. 
# The profile object sets the parametrization and properties of the profile function. 
# freeGSNKE accepts the same profile classes available in freeGS: ConstrainPaxisIp or ConstrainBetapIp.
# The profile profile object also sets the value of the total plasma current.
# Example use of a ConstrainPaxisIp profile object:
from freegsnke.jtor_update import ConstrainPaxisIp
profiles = ConstrainPaxisIp(# Equilibrium and limiter objects are passed to the profile to inform calculations relating to the limiter
                            eq, tokamak.limiter,
                            # The following set the desired values for the profile properties
                            8.1e3, # Plasma pressure on axis [Pascals]
                            6.2e5, # Total plasma current [Amps]
                            0.5, # vacuum f = R*Bt
                            # The following are the coefficients that define the peakedness of the plasma current density distribution
                            # see e.g. arXiv:1503.03135
                            alpha_m = 1.8,
                            alpha_n = 1.2)
# For an example use of a ConstrainBetapIp profile object, please un-comment the following:
# from freegsnke.jtor_update import ConstrainBetapIp
# profiles = ConstrainBetapIp(
#                               # Equilibrium and limiter objects are passed to the profile to inform calculations relating to the limiter:
#                               eq, tokamak.limiter,
#                               # The following set the desired values for the profile properties
#                               0.2, # value of beta_poloidal
#                               6.2e5, # Plasma current [Amps]
#                               0.5, # vacuum f = R*Bt
#                               # The following are the coefficients that define the peakedness of the plasma current density distribution
#                               # see e.g. arXiv:1503.03135
#                               alpha_m = 1.8,
#                               alpha_n = 1.2)

## Limited Equilibrium
### In this case, the plasma "touches" the limiter of the tokamak and is confined by the solid structures of the vessel

In [ ]:
eq.tokamak['P6'].current = 0.1
eq.tokamak['P6'].control = False
eq.tokamak['Solenoid'].control = False
eq.tokamak['Solenoid'].current = 15000
# Nonlinear solve
freegs.solve(eq,          # The equilibrium to adjust
             profiles,    # The plasma profiles
             constrain,   # Plasma control constraints
             show=False,
             rtol=3e-3)

eq.tokamak['Solenoid'].current = 40000
freegs.solve(eq,          # The equilibrium to adjust
             profiles,    # The plasma profiles
             constrain,   # Plasma control constraints
             show=False,
             rtol=3e-3) 

NK.solve(eq, profiles, target_relative_tolerance=1e-8)
fig = plt.figure(figsize=(5, 10), dpi=80);
ax = fig.add_subplot(111);
ax.grid(True,which='both');
eq.plot(axis=ax,show=False);
eq.tokamak.plot(axis=ax,show=False);
constrain.plot(axis=ax,show=False)

qprof = eq.q()
plt.figure()
plt.plot(qprof[0], qprof[1], '+')
plt.title('q profile')

In [ ]:

# Sets some shape constraints (here very close to those used for initialization)
Rx = 0.45
Zx = 1.18

Rmid = 1.4   # Outboard midplane
Rin = 0.28  # Inboard midplane

xpoints = [(Rx, -Zx-.01),   # (R,Z) locations of X-points
           (Rx,  Zx)]
isoflux = [
           (Rx,Zx, Rx,-Zx),
           (Rmid, 0, Rin, 0.0),
           (Rmid,0, Rx,Zx),
           (Rmid,0, 1.2,.7),
           (Rmid,0, 1.2,-.7),
    
           # Link inner and outer midplane locations
           (Rx, Zx, .85, 1.7),
           (Rx, Zx, .75, 1.6),
           (Rx, Zx, Rin, 0.2),
           (Rx, Zx, Rin, 0.1),
           (Rx,-Zx, Rin, -0.1),
           (Rx,-Zx, Rin, -0.2),
           (Rx,-Zx, .85, -1.7),
           (Rx,-Zx, .75, -1.6),

           (Rx,-Zx, 0.45, -1.8),
           (Rx, Zx, 0.45,  1.8),
           ]

eq.tokamak['P6'].current = 0
eq.tokamak['P6'].control = False
eq.tokamak['Solenoid'].control = False

constrain = freegs.control.constrain(xpoints=xpoints, 
                                     gamma=5e-6, 
                                     isoflux=isoflux
                                    )
constrain(eq)
                                    
# from freegsnke import GSstaticsolver
NK = GSstaticsolver.NKGSsolver(eq)     